In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from sqlalchemy import create_engine 

In [26]:
#Originally from kaggle: https://www.kaggle.com/rdoume/beerreviews
#This files contains around 1.5 Millions reviews of Beer from BeerAdvocates
beer_df = pd.read_csv('beer_reviews.csv')
beer_df.head(5)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [27]:
seperator = ' '
beer_df['brewery_beer']= beer_df['brewery_name'].astype(str) + seperator + beer_df['beer_name'].astype(str)
beer_df.head()



,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,brewery_beer
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986,Vecchio Birraio Sausa Weizen
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213,Vecchio Birraio Red Moon
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215,Vecchio Birraio Black Horse Black Beer
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969,Vecchio Birraio Sausa Pils
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883,Caldera Brewing Company Cauldron DIPA


In [33]:
#clean up data frame so we have only the necessary data
beer_merge_df = beer_df[['brewery_name', 'beer_name', 'brewery_beer', 'beer_style', 'beer_abv', 'review_overall']]
beer_merge_df.head()

,brewery_name,beer_name,brewery_beer,beer_style,beer_abv,review_overall
0,Vecchio Birraio,Sausa Weizen,Vecchio Birraio Sausa Weizen,Hefeweizen,5.0,1.5
1,Vecchio Birraio,Red Moon,Vecchio Birraio Red Moon,English Strong Ale,6.2,3.0
2,Vecchio Birraio,Black Horse Black Beer,Vecchio Birraio Black Horse Black Beer,Foreign / Export Stout,6.5,3.0
3,Vecchio Birraio,Sausa Pils,Vecchio Birraio Sausa Pils,German Pilsener,5.0,3.0
4,Caldera Brewing Company,Cauldron DIPA,Caldera Brewing Company Cauldron DIPA,American Double / Imperial IPA,7.7,4.0


# Scraping RateBeer

## Get List of Styles

In [16]:
#scrape RateBeer for list 
base_url = 'https://www.ratebeer.com'
response = requests.get(base_url+'/beerstyles/')
soup = BeautifulSoup(response.text, 'html.parser')

In [17]:
#get all of the list elements
styleGroups = soup.find_all('ul', class_="styleGroup")

In [19]:
#go through the style groupings
styles=[]
links=[]
for i in range(0,7):
    results = styleGroups[i].find_all('li')
    
    #loop through each style within the grouping
    for result in results:

        #error handling
        try:
            #Get Beer Styles
            beer_style=result.find('a').text
            #Get hrefs to those styles to scrape 
            link=result.a['href']

            #print only if it has a style and link 
            if (beer_style and link):
                styles.append(beer_style)
                links.append(link)
                print('-'*30)
                print(beer_style)
                print(link)
        except AttributeError as e:
            print(e)

------------------------------
Altbier / Sticke Alt
/beerstyles/altbier-sticke-alt/2/
------------------------------
BarleyWine / WheatWine / RyeWine
/beerstyles/barleywine-wheatwine-ryewine/11/
------------------------------
Bitter - Ordinary / Best
/beerstyles/bitter-ordinary-best/20/
------------------------------
Bitter - Premium / Strong / ESB
/beerstyles/bitter-premium-strong-esb/101/
------------------------------
Blonde Ale / Golden Ale
/beerstyles/blonde-ale-golden-ale/54/
------------------------------
Brown Ale
/beerstyles/brown-ale/15/
------------------------------
Brown Ale - Imperial
/beerstyles/brown-ale-imperial/131/
------------------------------
California Common / Steam Beer
/beerstyles/california-common-steam-beer/42/
------------------------------
Cream Ale
/beerstyles/cream-ale/35/
------------------------------
IIPA DIPA - Hazy / Double NEIPA
/beerstyles/iipa-dipa-hazy-double-neipa/145/
------------------------------
IIPA DIPA - Imperial / Double IPA
/beerstyles

## Get Top 50 List for Each Style

In [20]:
#create empty_df
master_df = pd.DataFrame(columns =['Rank','NAME', 'COUNT', 'ABV', 'SCORE', 'Style'])
#loop through all styles and get the top50 list

for style_link in links:
    beer_url = base_url+style_link
    driver =webdriver.Chrome('chromedriver')
    driver.get(beer_url)
    time.sleep(5)
    tables = pd.read_html(driver.page_source)
    beer_df=tables[0]
    beer_df['Style']= BeautifulSoup(driver.page_source, 'html').find('h1').text
    beer_df=beer_df.rename(columns={'Unnamed: 0': 'Rank'})
    master_df = pd.concat([master_df,beer_df])
    driver.quit()
master_df

,Rank,NAME,COUNT,ABV,SCORE,Style,0
0,1,Uerige Doppel Sticke,933,8.5%,3.76,Altbier / Sticke Alt,NaN
1,2,Schumacher 1838er,117,5.0%,3.71,Altbier / Sticke Alt,NaN
2,3,Schlüssel Stike,12,6.0%,3.67,Altbier / Sticke Alt,NaN
3,4,Godspeed Oberkassel,18,4.4%,3.63,Altbier / Sticke Alt,NaN
4,5,Vrooden Doppelsticke Altbier,16,8.5%,3.63,Altbier / Sticke Alt,NaN
...,...,...,...,...,...,...,...
45,46,Klostergården Alstadberger,99,6.5%,3.50,Traditional Ale - Other,NaN
46,47,WRCLW Schöps Smoked Plums Porto BA,27,4.8%,3.50,Traditional Ale - Other,NaN
47,48,Olvalde Brynhildr's Gift,51,8.2%,3.50,Traditional Ale - Other,NaN
48,49,Blaugies Bière Darbyste,563,5.8%,3.49,Traditional Ale - Other,NaN


In [24]:
#save to csv so I do not have to scrape data again
master_df.to_csv(index='false')
master_df.head()


,Rank,NAME,COUNT,ABV,SCORE,Style
0,1,Uerige Doppel Sticke,933,8.5%,3.76,Altbier / Sticke Alt
1,2,Schumacher 1838er,117,5.0%,3.71,Altbier / Sticke Alt
2,3,Schlüssel Stike,12,6.0%,3.67,Altbier / Sticke Alt
3,4,Godspeed Oberkassel,18,4.4%,3.63,Altbier / Sticke Alt
4,5,Vrooden Doppelsticke Altbier,16,8.5%,3.63,Altbier / Sticke Alt


# Move data to SQLite files

In [34]:
beer_merge_df
master_df

#create connection for beer advocate data 
engine = create_engine('sqlite:///beer_advocate_2017.sqlite', echo=False)
sqlite_connection = engine.connect()

beer_merge_df.to_sql("beer_advocate_2017_ratings", sqlite_connection)
sqlite_connection.close()


In [35]:
#create connection for RateBeer Scraped Data
engine = create_engine('sqlite:///RateBeer_scrape.sqlite', echo=False)
sqlite_connection = engine.connect()

master_df.to_sql("beer_advocate_2017_ratings", sqlite_connection)
sqlite_connection.close()